## Exemplo de otimização com thread

## versão single thread

In [1]:
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

start = time.time()
countdown(COUNT)
end = time.time()

print('Time -', end - start)


Time - 2.1578071117401123


## versão multi thread

In [2]:
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print('Time  -', end - start)

Time  - 7.298934698104858


## paralelizando predições

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.datasets.mnist as mnist

from timeit import default_timer as timer
import numpy as np

import threading

In [4]:
# Preparação para CNN
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
x_train = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
x_test = test_images.astype('float32') / 255

y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

# executando uma predição

In [5]:


start = timer()

new_model = tf.keras.models.load_model('MnistModel.h5')
new_model.summary()
res=new_model.predict(train_images)
#print(res.shape)

end = timer()
print('tempo total: ',end - start)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

# executando diversas predições

In [7]:
new_model = tf.keras.models.load_model('MnistModel.h5')
new_model.summary()



start = timer()

for i in range(10):
    rand=train_images[np.random.randint(train_images.shape[0], size=50000), :,:,:]
    res=new_model.predict(rand)
    print(res.shape)

end = timer()
print('tempo total: ',end - start)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

# paralelizando execuções em threads (1 por thread)

In [10]:
def thread_function(name):
    print(name)
    rand=train_images[np.random.randint(train_images.shape[0], size=50000), :,:,:]
    res=new_model.predict(rand)
    print(res.shape)

In [11]:
start = timer()

for i in range(10):
    x = threading.Thread(target=thread_function,args=(i,))
    x.start()
x.join()

end = timer()
print('tempo total: ',end - start)

0
1
2
3
4
5
6
7
89

(50000, 10)
tempo total:  10.858237707987428
(50000, 10)
(50000, 10)
(50000, 10)
(50000, 10)
(50000, 10)
(50000, 10)
(50000, 10)
(50000, 10)
(50000, 10)


# paralelizando execuções em threads (3 por thread)

In [13]:
def thread_function(name):
    if (name==2):
        am=4
    else:
        am=3
    for i in range(am):
        print(name)
        rand=train_images[np.random.randint(train_images.shape[0], size=50000), :,:,:]
        res=new_model.predict(rand)
        print(res.shape)

In [14]:
start = timer()

for i in range(3):
    x = threading.Thread(target=thread_function,args=(i,))
    x.start()
x.join()

end = timer()
print('tempo total: ',end - start)

0
1
2
(50000, 10)
0
(50000, 10)
2
(50000, 10)
1
(50000, 10)
2
(50000, 10)
0
(50000, 10)
1
(50000, 10)
(50000, 10)
(50000, 10)
2
(50000, 10)
tempo total:  12.488636401947588
